In [127]:
import torch
from torchmetrics.classification import MultilabelAccuracy
from torchmetrics.functional.classification import multilabel_accuracy, multilabel_f1_score
from torchmetrics.functional.classification import multilabel_recall, multilabel_precision
from torchmetrics.functional.classification import multiclass_auroc, multilabel_auroc
from torchmetrics.functional.classification import multilabel_exact_match

## Multi-label

In [131]:
preds = torch.tensor(
    [
        [1, 0, 1, 0],
        [1, 1, 0, 0]
    ]
)

target = torch.tensor(
    [
        [1, 0, 1, 0],
        [0, 1, 1, 1]S
    ]
)

# for individual samples
TP, TN = 2, 2
FP, FN = 2, 2

multilabel_exact_match(preds, target, num_labels=4)

tensor(0.5000)

In [132]:
multilabel_accuracy(preds, target, num_labels=4, average='micro')

tensor(0.6250)

In [96]:
r = multilabel_recall(preds, target, num_labels=4)
r, TP/(TP+FN)

(tensor(0.3750), 0.5)

In [95]:
p = multilabel_precision(preds, target, num_labels=4,)
p, TP/(TP+FP)

(tensor(0.3750), 0.5)

In [97]:
f1 = multilabel_f1_score(preds, target, num_labels=4)
f1, 2*(p*r)/(p+r)

(tensor(0.3333), tensor(0.3750))

In [63]:
2*(p*r)/(p+r)

tensor(0.3750)

In [86]:
from sklearn.metrics import f1_score, precision_score, recall_score
f1_score(target.detach(), preds.detach(), average=None)

array([0.66666667, 0.        , 0.66666667, 0.        ])

In [76]:
precision_score(target.detach(), preds.detach(), average='macro')

c:\users\spetr\documents\github\atel_speciale\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.375

In [77]:
recall_score(target.detach(), preds.detach(), average='macro')

c:\users\spetr\documents\github\atel_speciale\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.375

In [126]:
preds = torch.tensor([[0.75, 0.05, 0.35],
                      [0.45, 0.75, 0.05],
                      [0.05, 0.55, 0.75],
                      [0.05, 0.65, 0.05]])
target = torch.tensor([[1, 0, 1],
                       [0, 0, 0],
                       [0, 1, 1],
                       [1, 1, 1]])
multilabel_auroc(preds, target, num_labels=3, average="macro", thresholds=None)

tensor(0.6528)

In [124]:
multilabel_auroc(preds, target, num_labels=3, average=None, thresholds=None).mean()

tensor(0.6528)

## Multi-class

In [111]:
from torchmetrics.functional.classification import multiclass_accuracy, multiclass_f1_score
from torchmetrics.functional.classification import multiclass_recall, multiclass_precision
from torchmetrics.functional.classification import multiclass_accuracy
from torchmetrics.functional import accuracy

In [100]:
target = torch.tensor([2, 1, 0, 0])
preds = torch.tensor([
  [0.16, 0.26, 0.58],
  [0.22, 0.61, 0.17],
  [0.71, 0.09, 0.20],
  [0.05, 0.82, 0.13],
])

In [112]:
accuracy(preds, target)

tensor(0.7500)

In [119]:
multiclass_accuracy(preds, target, num_classes=3, average='none').mean()

tensor(0.8333)

In [110]:
# 2*(p*r)/(p+r)
multiclass_f1_score(preds, target, num_classes=3, average='none')

tensor([0.6667, 0.6667, 1.0000])

In [109]:
# TP/(TP+FP)
multiclass_precision(preds, target, num_classes=3, average='none')

tensor([1.0000, 0.5000, 1.0000])

In [108]:
# TP/(TP+FN)
multiclass_recall(preds, target, num_classes=3, average='none')

tensor([0.5000, 1.0000, 1.0000])